In [1]:
%load_ext autoreload
%autoreload 2
import configparser
import psycopg2
from sql_queries import create_table_queries, drop_table_queries,insert_table_queries,copy_table_queries

In [2]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

#### Code to examine the file

In [86]:
import boto3
import json
#Code to get the first object
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=config['AWS']['KEY'],
                       aws_secret_access_key=config['AWS']['SECRET']
                   )

#First bucket
s3Bucket=  s3.Bucket("udacity-dend")
#get 1 object
FirstObject =[obj for obj in s3Bucket.objects.filter(Prefix="log_data")][1]
s3_clientobj = s3.Object(bucket_name=FirstObject.bucket_name, key=FirstObject.key)
s3_clientdata = s3_clientobj.get()['Body'].read().decode('utf-8')


print("printing s3_clientdata")
print(s3_clientdata)

printing s3_clientdata
{"artist":null,"auth":"Logged In","firstName":"Walter","gender":"M","itemInSession":0,"lastName":"Frye","length":null,"level":"free","location":"San Francisco-Oakland-Hayward, CA","method":"GET","page":"Home","registration":1540919166796.0,"sessionId":38,"song":null,"status":200,"ts":1541105830796,"userAgent":"\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.143 Safari\/537.36\"","userId":"39"}
{"artist":null,"auth":"Logged In","firstName":"Kaylee","gender":"F","itemInSession":0,"lastName":"Summers","length":null,"level":"free","location":"Phoenix-Mesa-Scottsdale, AZ","method":"GET","page":"Home","registration":1540344794796.0,"sessionId":139,"song":null,"status":200,"ts":1541106106796,"userAgent":"\"Mozilla\/5.0 (Windows NT 6.1; WOW64) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/35.0.1916.153 Safari\/537.36\"","userId":"8"}
{"artist":"Des'ree","auth":"Logged In","firstName":"Kaylee","gender":"F","ite

In [90]:
import boto3
import json
#Code to get the first object
s3 = boto3.resource('s3',
                       region_name="us-west-2",
                       aws_access_key_id=config['AWS']['KEY'],
                       aws_secret_access_key=config['AWS']['SECRET']
                   )

#First bucket
s3Bucket=  s3.Bucket("udacity-dend")
#get 1 object
FirstObject =[obj for obj in s3Bucket.objects.filter(Prefix="song_data/A/A/A")][1]
s3_clientobj = s3.Object(bucket_name=FirstObject.bucket_name, key=FirstObject.key)
s3_clientdata = s3_clientobj.get()['Body'].read().decode('utf-8')


print("printing s3_clientdata")
print(s3_clientdata)

printing s3_clientdata
{"artist_id":"AR73AIO1187B9AD57B","artist_latitude":37.77916,"artist_location":"San Francisco, CA","artist_longitude":-122.42005,"artist_name":"Western Addiction","duration":118.07302,"num_songs":1,"song_id":"SOQPWCR12A6D4FB2A3","title":"A Poor Recipe For Civic Cohesion","year":2005}


#### Run the py file

In [18]:
%%bash
python create_tables.py

In [19]:
%%bash
python etl.py


    copy stg_raw_events from 's3://udacity-dend/log_data'
    credentials 'aws_iam_role=arn:aws:iam::009709556450:role/dwhRole'
    format as json 's3://udacity-dend/log_json_path.json'
    STATUPDATE ON region 'us-west-2';

time used 1.9946722984313965 s

    copy stg_raw_songs from 's3://udacity-dend/song_data/A/A'
    credentials 'aws_iam_role=arn:aws:iam::009709556450:role/dwhRole'
    format as json 'auto'
    STATUPDATE ON region 'us-west-2';

time used 8.157477855682373 s

INSERT INTO fdn_fact_songplays (start_time,user_id,level,song_id,artist_id,session_id,location,user_agent)
select  TIMESTAMP 'epoch' + ts/1000 *INTERVAL '1 second' as start_time
        ,userId,level,song_id,artist_id, sessionId, location, userAgent
                            from stg_raw_events A 
                            join (select distinct song_id, title  from stg_raw_songs) B on A.song=B.title
                            join (select distinct artist_id,artist_name from stg_raw_songs) C on A.artist=C